In [1]:
from hdbcli import dbapi

In [2]:
connection = dbapi.connect('52.211.228.133', 39015, 'SYSTEM', 'Glorp2018!')
connection.isconnected()

In [11]:
cursor = connection.cursor()
cursor.execute('select top 10 * from SYSTEM.CMPL100K')
for row in cursor:
    print(row)

True